<a href="https://colab.research.google.com/github/Hoangnhat4869/TrafficLightDetection/blob/main/TrafficLightTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. Install YOLOv5 and some requirements

In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16567, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 16567 (delta 23), reused 31 (delta 13), pack-reused 16522
Receiving objects: 100% (16567/16567), 15.02 MiB | 19.57 MiB/s, done.
Resolving deltas: 100% (11382/11382), done.


In [ ]:
import shutil

In [ ]:
shutil.rmtree("yolov5")

In [ ]:
cd yolov5

/content/yolov5/yolov5


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.1/752.1 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_

### 2. Train model

In [ ]:
!pip install pybboxes
import xml.etree.ElementTree as ET
import os
import shutil
from tqdm import tqdm
import pandas as pd
import numpy as np
import pandas as pd
import pybboxes as pbx
from collections import defaultdict

In [ ]:
# create output path
output_path = '/content/yolov5'
# create train, val folders in data directory of YOLOv5
train_dir = output_path + '/data/train'
val_dir = output_path + '/data/valid'

if os.path.exists(train_dir): shutil.rmtree(train_dir)
if os.path.exists(val_dir): shutil.rmtree(val_dir)

os.mkdir(train_dir)
os.mkdir(val_dir)

# train, val each containing images and labels folders
os.mkdir(train_dir + '/images')
os.mkdir(train_dir + '/labels')
os.mkdir(val_dir + '/images')
os.mkdir(val_dir + '/labels')

In [ ]:
## copy traffic light images and labels to train and val folders
input_path = '/content/drive/MyDrive/TrafficLightDetection/dataset'

images_train_path = input_path + '/train/images'
labels_train_path = input_path + '/train/labels'

images_val_path = input_path + '/valid/images'
labels_val_path = input_path + '/valid/labels'

for file in os.listdir(images_train_path):
  shutil.copy(images_train_path + '/' + file, train_dir + '/images')

for file in os.listdir(labels_train_path):
  shutil.copy(labels_train_path + '/' + file, train_dir + '/labels')

for file in os.listdir(images_val_path):
  shutil.copy(images_val_path + '/' + file, val_dir + '/images')

for file in os.listdir(labels_val_path):
  shutil.copy(labels_val_path + '/' + file, val_dir + '/labels')

In [ ]:
# cd ../drive/MyDrive/TrafficLightDetection/

In [ ]:
# ## Because of the different datasets, we need to relabel the data.
# import os

# def changefile(path):
#     with open(path, 'r') as file:
#         content = file.readlines()
#     newcontent = []
#     for c in content:
#         newidx = int(c[0:1]) + 14
#         c = str(newidx) + c[1:]
#         newcontent.append(c)
#     with open(path, 'w') as file:
#         file.writelines(newcontent)

# path = os.getcwd() + '/signdataset/labels'
# for file in os.listdir(path + '/train'):
#     changefile(path + '/train/' + file)

# for file in os.listdir(path + '/val'):
#     changefile(path + '/val/' + file)

In [ ]:
## copy sign images and labels to train and val folders
sign_input_path = '/content/drive/MyDrive/TrafficLightDetection/signdataset'

sign_images_train_path = sign_input_path + '/images/train'
sign_labels_train_path = sign_input_path + '/labels/train'

sign_images_val_path = sign_input_path + '/images/val'
sign_labels_val_path = sign_input_path + '/labels/val'

for file in os.listdir(sign_images_train_path):
  shutil.copy(sign_images_train_path + '/' + file, train_dir + '/images')

for file in os.listdir(sign_labels_train_path):
  shutil.copy(sign_labels_train_path + '/' + file, train_dir + '/labels')

for file in os.listdir(sign_images_val_path):
  shutil.copy(sign_images_val_path + '/' + file, val_dir + '/images')

for file in os.listdir(sign_labels_val_path):
  shutil.copy(sign_labels_val_path + '/' + file, val_dir + '/labels')

In [ ]:
classes = ['Green Light', 'Red Light', 'Speed Limit 10', 'Speed Limit 100', 'Speed Limit 110', 'Speed Limit 120', 'Speed Limit 20', 'Speed Limit 30', 'Speed Limit 40', 'Speed Limit 50', 'Speed Limit 60', 'Speed Limit 70', 'Speed Limit 80', 'Speed Limit 90', 'Stop', 'No pedestrian crossing', 'No parking', 'Parking', 'Give way', 'One way', 'No left turn', 'Speed limit', 'Bus lane', 'Pedestrian crossing']
with open(f'{output_path}/data/data.yaml', 'w') as f:
  f.write('train: ./data/train/images\n')
  f.write('val: ./data/valid/images\n')
  f.write('nc: 24\n')
  f.write(f'names: {classes}')

In [ ]:
epochs = 20
!python train.py --img 640 --batch 16 --epochs {epochs} --data data.yaml --weights yolov5s.pt

2024-04-27 05:36:11.914093: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 05:36:11.914142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 05:36:12.023563: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optim

### 3. Store weights and test

In [ ]:
import shutil
import os
os.mkdir("/content/drive/MyDrive/TrafficLightDetection/weights")
shutil.copy('/content/yolov5/runs/train/exp/weights/best.pt', '/content/drive/MyDrive/TrafficLightDetection/weights')

In [ ]:
!python detect.py --weights /content/drive/MyDrive/TrafficLightDetection/best.pt --img 416 --conf 0.4 --source /content/drive/MyDrive/TrafficLightDetection/test_images/test.jpg

detect: weights=['/content/drive/MyDrive/TrafficLightDetection/best.pt'], source=/content/drive/MyDrive/TrafficLightDetection/test_images/test.jpg, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-304-g22361691 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7050580 parameters, 0 gradients, 15.9 GFLOPs
image 1/1 /content/drive/MyDrive/TrafficLightDetection/test_images/test.jpg: 288x416 2 Red Lights, 217.7ms
Speed: 1.5ms pre-process, 217.7ms inference, 31.9ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp
